In [4]:
import unittest

In [5]:
import sys
sys.path.append("/home/till/Desktop/GitHub/self-driving-car/")
sys.path.append("/home/till/Desktop/GitHub/self-driving-car/src")

In [6]:
from __future__ import annotations
from typing import Dict, List, Union

import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

import gymnasium as gym

# suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium.spaces.box") # module="gymnasium"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import src
from src.actor_critic_discrete import DiscreteActorCritic
from src.actor_critic_dreamer import ActorCriticDreamer
from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import make_imagination_env
from src.mlp import MLP
from src.preprocessing import transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np, symlog, symexp, twohot_encode, ExponentialMovingAvg,
                       ActionExponentialMovingAvg, MetricsTracker)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

In [27]:
class TestAddition(unittest.TestCase):
    
    def addition(self,x,y):
        return x+y
    
    def test_addition(self):
        self.assertEqual(self.addition(1,2), 3)

In [19]:
# if __name__ == "__main__":
#     unittest.main()

hi


In [30]:
agent = src.actor_critic_discrete.DiscreteActorCritic()

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [31]:
agent.critic

MLP(
  (layers): Sequential(
    (layer_0): Sequential(
      (0): Linear(in_features=1536, out_features=256, bias=True)
      (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (2): SiLU(inplace=True)
    )
    (layer_1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (2): SiLU(inplace=True)
    )
    (layer_2): Sequential(
      (0): Linear(in_features=256, out_features=255, bias=True)
      (2): Softmax(dim=-1)
    )
  )
)

In [45]:
sample_input = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])

In [47]:
value_pred, critic_dist = agent.apply_critic(sample_input)

In [50]:
len(value_pred.shape)

0

In [57]:
critic_dist.shape

torch.Size([255])

In [59]:
config["num_buckets"]

255

In [63]:
torch.Size([config["num_buckets"]])

torch.Size([255])

In [51]:
class TestDiscreteActorCritic(unittest.TestCase):
    
    agent = src.actor_critic_discrete.DiscreteActorCritic()
    
    # test if the critic softmax sums to 1
    # test the critic output shape 
    # => for a single instance
    # => for a batch
    def test_critic(self):
        
        sample_input = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])
        value_pred, critic_dist = agent.apply_critic(sample_input)
        
        # value_pred should be a scalar without shape
        self.assertEqual(len(value_pred.shape), 0)

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [65]:
sample_batch = torch.randn(32, config["H"] + config["Z"]).to(config["device"])
sample_batch.shape

torch.Size([32, 1536])

In [67]:
value_pred, critic_dist = agent.apply_critic(sample_batch)

In [72]:
torch.sum(critic_dist, dim=-1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0', grad_fn=<SumBackward1>)

In [8]:
rssm = RSSM().to(device)

Initializing encoder:
- adding ConvBlock((3, 64))                   ==> output shape: (64, 32, 32) ==> prod: 65536
- adding ConvBlock((64, 128))                   ==> output shape: (128, 16, 16) ==> prod: 32768
- adding ConvBlock((128, 256))                   ==> output shape: (256, 8, 8) ==> prod: 16384
- adding ConvBlock((256, 512))                   ==> output shape: (512, 4, 4) ==> prod: 8192
- adding ConvBlock((512, 256))                   ==> output shape: (256, 2, 2) ==> prod: 1024
- adding Flatten()
- adding Reshape: (*,1024) => (*,32,32)

Initializing decoder:
- adding Reshape: (*,1024) => (*,256,2,2)
- adding transpose ConvBlock(256, 256)                   ==> output shape: (256, 4, 4) ==> prod: 4096
- adding transpose ConvBlock(256, 512)                   ==> output shape: (512, 8, 8) ==> prod: 32768
- adding transpose ConvBlock(512, 256)                   ==> output shape: (256, 16, 16) ==> prod: 65536
- adding transpose ConvBlock(256, 128)                   ==> output shap

In [21]:
action = torch.randn(config["A"])
h = torch.randn(config["H"]).to(config["device"]).unsqueeze(0)
z = torch.randn(config["Z"]).to(config["device"]).unsqueeze(0)
rssm.step(action, h, z)

(tensor([[ 1.0907e-01,  3.7030e-01,  8.2652e-02,  1.4333e-01,  5.7968e-02,
           5.5485e-01, -2.6533e-01, -1.6376e+00,  1.7660e-01,  2.8212e-01,
          -4.4095e-01,  4.1365e-01,  3.2652e-01,  7.4823e-01,  8.1275e-01,
           3.7246e-01, -5.4447e-01,  8.5899e-02,  5.5583e-02,  1.1376e+00,
          -7.5834e-01,  2.5150e-01,  9.1271e-01,  1.2435e+00,  6.6928e-01,
           6.1936e-01,  2.4938e-01,  8.5445e-01,  3.3247e-01, -6.8071e-02,
           2.1838e-01,  7.0267e-01,  1.2314e+00,  1.3962e-01,  5.0053e-01,
          -3.3483e-01,  4.7302e-01,  8.1347e-01,  7.1268e-01,  8.0030e-01,
          -2.0715e-01,  8.8734e-01, -1.0655e+00, -6.5527e-01,  2.3202e-01,
           2.7707e-01,  1.0166e+00,  1.7675e+00,  8.7402e-01, -3.1712e-01,
          -3.5289e-01, -5.2366e-01, -1.0398e-01,  1.9573e-02,  8.4406e-02,
           4.0108e-01,  7.7255e-01, -6.4351e-01, -1.0140e-01,  4.9053e-01,
          -8.2503e-02,  3.4494e-01, -3.5788e-02, -7.9148e-02,  7.5272e-01,
           4.1834e-01,  9

In [19]:
print("action:", action.shape)
print("h:", h.shape)
print("z:", z.shape)

action: torch.Size([3])
h: torch.Size([512])
z: torch.Size([1024])
